# Extracting Data from Quantrocket

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from quantrocket.license import set_license
set_license("2b416b15-de0e-11ee-b542-958e78a02bc9")

In [ ]:
from quantrocket.master import collect_usstock_listings
collect_usstock_listings()

In [ ]:
from quantrocket.master import get_securities
securities = get_securities(symbols='AAPL', fields=['Sid', 'Symbol', 'Exchange'], vendors='usstock')
securities

In [ ]:
from quantrocket.zipline import create_usstock_bundle
create_usstock_bundle("usstock-free-1min",free = True)

In [ ]:
from quantrocket.zipline import ingest_bundle
ingest_bundle("usstock-free-1min")

In [ ]:
from quantrocket import get_prices
data = get_prices("usstock-free-1min", data_frequency='daily', sids="FIBBG000B9XRY4", start_date='2023-01-01', end_date='2023-12-31', fields=["Open","High","Low","Close"])

In [ ]:
df = data.reset_index()

In [ ]:
high = df[df['Field'] == 'High']
high = list(high['FIBBG000B9XRY4'])

In [ ]:
low = df[df['Field'] == 'Low']
low = list(low['FIBBG000B9XRY4'])

In [ ]:
open = df[df['Field'] == 'Open']
open = list(open['FIBBG000B9XRY4'])

In [ ]:
df = df[df['Field'] == 'Close']

In [ ]:
df['Open'] = open
df['High'] = high
df['low'] = low

In [ ]:
df.rename(columns={'FIBBG000B9XRY4': 'Close'}, inplace=True)

In [ ]:
df.to_csv("/content/drive/MyDrive/Stock_data.csv", index=False)

# Import Libraries

In [60]:
import warnings
import numpy as np
import pandas as pd
from math import sqrt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
warnings.filterwarnings("ignore")

# Reading Data

In [ ]:
# Read the CSV file
df = pd.read_csv("/content/drive/MyDrive/Stock_data.csv")

In [ ]:
# Drop the column
df = df.drop("Field", axis=1)

In [ ]:
df

,Date,Close,Open,High,low
0,2023-01-03,124.375,129.556,130.172,123.480
1,2023-01-04,125.658,126.185,127.941,124.385
2,2023-01-05,124.325,126.423,127.060,124.067
3,2023-01-06,128.900,125.310,129.566,124.196
4,2023-01-09,129.427,129.740,132.668,129.168
...,...,...,...,...,...
245,2023-12-22,193.600,195.180,195.410,192.970
246,2023-12-26,193.050,193.610,193.890,192.830
247,2023-12-27,193.150,192.490,193.500,191.090
248,2023-12-28,193.580,194.140,194.660,193.170


# Model

**Stock Trading Decision Model Inference:**

This Python class, `Model`, represents a simple model for making trading decisions based on stock price movements. Here's an overview of its functionality:

1. **Initialization:**
   - The class initializes with default values for portfolio value (`value`), current state (`state`), optimal buy indices (`optimal_buy_indicies`), and transition counts (`transition_counts`).

2. **Return Calculation:**
   - The `calculate_returns` method computes the returns based on the current and previous prices.

3. **Portfolio Value Update:**
   - The `update_portfolio_value` method updates the portfolio value based on the trading decision.

4. **Transition Counts and Probabilities:**
   - The `update_transition_counts` method updates the transition counts between different states.
   - The `calculate_transition_probabilities` method computes transition probabilities based on transition counts.

5. **Trading Decision:**
   - The `make_trading_decision` method makes trading decisions based on transition probabilities and current state.
   - If the probability of price increase is higher, it suggests buying the stock; otherwise, it suggests not buying.

**Stock Trading Decision Procedure:**

This procedure outlines the steps involved in making trading decisions based on stock price movements:

1. **Calculate Returns:**
   - Calculate the returns based on the formula
        r(d) = (p(d) - p(d-1))/p(d-1), where p(d)  represents the price on day d .

2. **Classify States:**
   - Classify states based on the threshold:
     - If r(d) > 0.01 , s(d) = +1 (Bull).
     - Else if  -0.1 < r(d) < 0.1 , s(d) = 0  (Flat).
     - Else, s(d) = -1  (Bear).

3. **Calculate Transition Probability Matrix:**
   - Calculate the transition probability matrix in a streaming fashion until the previous state.
   - Use transition likelihoods to the additive state (positive price movement) and the reductive state (negative price movement) for decision-making.

4. **Make Trading Decision:**
   - If the likelihood of transitioning to the additive state is greater, buy the stock.

5. **Update Portfolio Value:**
   - When deciding to buy the stock on a particular day:
     - If s(d+1) = 1  and s(d) = 0 , update portfolio value as    
      V(d+1) = V(d) + 1 .
     - Else if  s(d+1) = -1  and s(d) = 0 , update portfolio value as   V(d+1) = V(d) - 1 .

6. **Increase Transition Count:**
   - After updating the portfolio value, increase the transition count for the current state transition.




In [66]:
class Model:
    def __init__(self):
        # Initialize model parameters
        self.value = 0  # Portfolio value
        self.state = 0  # Current market state: -1 (Bear), 0 (Flat), 1 (Bull)
        self.optimal_buy_indicies = []  # Indices where buying is optimal
        # Transition counts between different states
        self.transition_counts = {
            -1: {-1: 0, 0: 0, 1: 0},
            0: {-1: 0, 0: 0, 1: 0},
            1: {-1: 0, 0: 0, 1: 0}
        }

    # Calculate returns based on current and previous prices
    def calculate_returns(self, current_price, prev_price):
        return (current_price - prev_price) / prev_price

    # Update portfolio value based on trading decision and state transition
    def update_portfolio_value(self, decision, prev_state, current_state):
        if prev_state == 0 and current_state == 1:  # Moving from Flat to Bull
            self.value += 1
        elif prev_state == 0 and current_state == -1:  # Moving from Flat to Bear
            self.value -= 1

    # Update transition counts between states
    def update_transition_counts(self, prev_state, current_state):
        self.transition_counts[prev_state][current_state] += 1

    # Calculate transition probabilities based on transition counts
    def calculate_transition_probabilities(self):
        transition_probabilities = {}
        for prev_state, transitions in self.transition_counts.items():
            total_transitions = sum(transitions.values())
            if total_transitions == 0:  # Skip if no transitions recorded
                continue
            transition_probabilities[prev_state] = {
                next_state: count / total_transitions
                for next_state, count in transitions.items()
            }
        return transition_probabilities

    # Make trading decision based on current and previous prices
    def make_trading_decision(self, current_price, prev_price, count):
        returns = self.calculate_returns(current_price, prev_price)

        # Determine current market state
        if returns >= 0.01:
            current_state = 1  # Bull
        elif returns > -0.01 and returns < 0.01:
            current_state = 0  # Flat
        else:
            current_state = -1  # Bear

        prev_state = self.state
        # Calculate transition probabilities
        transition_probabilities = self.calculate_transition_probabilities()
        decision = "Don't Buy"  # Default decision

        # Decision making based on transition probabilities
        if self.state in transition_probabilities:
            prob_increase = transition_probabilities[self.state].get(1, 0)
            prob_decrease = transition_probabilities[self.state].get(-1, 0)

            if prob_increase > prob_decrease:  # Favor buying if probability of increase is higher
                decision = "You can Buy"
                self.optimal_buy_indicies.append(count)  # Record index where buying is optimal
                self.update_portfolio_value(decision, prev_state, current_state)
            elif prob_increase < prob_decrease:  # Don't buy if probability of decrease is higher
                decision = "Don't Buy"
        else:
            decision = "Don't Buy"

        # Update transition counts and current state
        self.update_transition_counts(self.state, current_state)
        self.state = current_state

        return decision


# Fitting the Model for the Given Dataset

In [68]:
# Example usage:
prices =  list(df['Close'])
portfolio = Model()
optimal_buy_dates = []

for i in range(1, len(prices)):
    decision = portfolio.make_trading_decision(prices[i], prices[i-1], i)
    print(f"Day {i+1}: Decision: {decision}, Portfolio Value: {portfolio.value}")


print("Optimal Buy Indicies :", portfolio.optimal_buy_indicies)
for index in portfolio.optimal_buy_indicies:
    optimal_buy_dates.append(df.iloc[index]['Date'])

print("Optimal Buy Dates    :", optimal_buy_dates)

TPM = portfolio.calculate_transition_probabilities()

states = ['Bear', 'Flat', 'Bull']
i = 0

for row in range(-1, 2):
  print(states[i], end="   ")
  i += 1
  for val in range(-1, 2):
    print(f"{TPM[val][row]:.4f}", end=" ")
  print(end="\n")

print("       ", end=" ")
for i in states:
  print(i, end="   ")
print(end="\n")

prob_increase = TPM[portfolio.state].get(1, 0)
prob_decrease = TPM[portfolio.state].get(-1, 0)

if prob_increase > prob_decrease:
  print(f"Day {len(df)+1}: Decision: You can Buy")
else:
  print(f"Day {len(df)+1}: Decision: Don't Buy")



Day 2: Decision: Don't Buy, Portfolio Value: 0
Day 3: Decision: Don't Buy, Portfolio Value: 0
Day 4: Decision: Don't Buy, Portfolio Value: 0
Day 5: Decision: Don't Buy, Portfolio Value: 0
Day 6: Decision: You can Buy, Portfolio Value: 0
Day 7: Decision: You can Buy, Portfolio Value: 1
Day 8: Decision: Don't Buy, Portfolio Value: 1
Day 9: Decision: You can Buy, Portfolio Value: 2
Day 10: Decision: Don't Buy, Portfolio Value: 2
Day 11: Decision: You can Buy, Portfolio Value: 2
Day 12: Decision: You can Buy, Portfolio Value: 2
Day 13: Decision: You can Buy, Portfolio Value: 3
Day 14: Decision: Don't Buy, Portfolio Value: 3
Day 15: Decision: Don't Buy, Portfolio Value: 3
Day 16: Decision: You can Buy, Portfolio Value: 3
Day 17: Decision: You can Buy, Portfolio Value: 4
Day 18: Decision: You can Buy, Portfolio Value: 4
Day 19: Decision: You can Buy, Portfolio Value: 4
Day 20: Decision: You can Buy, Portfolio Value: 4
Day 21: Decision: You can Buy, Portfolio Value: 4
Day 22: Decision: You ca